# TASHO - A model predictive control toolchain for constraint-based task specification of robot motions

#### Presented by: Alejandro Astudillo and Ajay Sathya

Tasho stands for “**TA**sk **S**pecification with receding **HO**rizon control”


First, we need to import the relevant modules from Tasho and other dependencies (casadi, numpy)

In [ ]:
import os
os.chdir(str(os.getcwd())+"/../../")


from tasho import robot as rob
from tasho import discrete_plan as dp

from tasho import problem_template as pt
from tasho import environment as env

import casadi as cs
from casadi import pi, cos, sin
from rockit import MultipleShooting, Ocp
import numpy as np

## Instantiate plan

In [ ]:
plan = dp.DiscretePlan()

## Set robot

In [ ]:
# Initial joint configuration
q0_val = [0, -0.523598, 0, 2.51799, 0, -0.523598, 0]
# Initial joint velocity
q_dot0_val = [0]*7

robot = rob.Robot('kinova')
robot.set_joint_acceleration_limits(lb = -30*3.14159/180, ub = 30*3.14159/180)
robot.set_state(q0_val + q_dot0_val)
robot.set_robot_input_resolution("acceleration")

## Set environment

In [ ]:
environment = env.Environment()

cube1 = env.Cube(length = 1, position = [0.5, 0, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = "/models/objects/cube_small.urdf")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf = "/models/objects/table.urdf")
table2 = env.Box(height = 0.3, position = [0,0.5, 0], orientation = [0.0, 0.0, 0.0, 1.0], urdf = "/models/objects/table.urdf")
# table3 = env.Box(height = 0.3, position = [0,-0.5, 0], orientation = [0.0, 0.0, 0.0, 1.0], urdf = "/models/objects/table.urdf")

In [ ]:
environment.add_object(cube1, "cube")
environment.add_object(table1, "table1")
environment.add_object(table2, "table2")
# environment.add_object(table3, "table3")

In [ ]:
environment.print_objects()

In [ ]:
plan.add_environment(environment)

## Task 1: Approximation to object

In [ ]:
horizon_size = 10
t_mpc = 0.5

T_goal = np.array([[0, 1, 0, 0.5], [1, 0, 0, 0], [0, 0, -1, 0.25], [0, 0, 0, 1]])  # T_goal = np.array([[0.0, 0., -1., 0.5], [0., 1., 0., 0.], [1.0, 0., 0.0, 0.5], [0.0, 0.0, 0.0, 1.0]]) # T_goal = np.array([[0., 0., -1., 0.5], [-1., 0., 0., 0.], [0., 1., 0.0, 0.5], [0.0, 0.0, 0.0, 1.0]]) # T_goal = np.array([[0., 1., 0., 0.5], [1., 0., 0., 0.], [0., 0., -1.0, 0.5], [0.0, 0.0, 0.0, 1.0]]) # T_goal = np.array([[0, 1, 0, 0], [1, 0, 0, -0.5], [0, 0, -1, 0.5], [0, 0, 0, 1]])

approach_task = pt.Point2Point(horizon_size*t_mpc, horizon_steps = horizon_size, goal = T_goal)

approach_task.add_robot(robot)

## Task 2: Picking the object up

In [ ]:
horizon_size = 30
t_mpc = 0.5

# T_goal = np.array([[-1, 0, 0, 0], [0, 1, 0, 0.5], [0, 0, -1, 0.25], [0, 0, 0, 1]])
# T_goal = np.array([[0, 0, 1, 0.5], [1, 0, 0, 0.2], [0, 1, 0, 0.5], [0, 0, 0, 1]])
T_goal = np.array([[0, 1, 0, 0.4], [1, 0, 0, 0.15], [0, 0, -1, 0.25], [0, 0, 0, 1]])

pickup_task = pt.Point2Point(time = horizon_size*t_mpc, horizon_steps = horizon_size, goal = T_goal)

pickup_task.add_robot(robot)

## Define discrete plan

In [ ]:
plan.add_task(approach_task, "approach")
plan.add_task(pickup_task, "pickup")

In [ ]:
plan.print_tasks()

## Solve specific tasks in the plan

In [ ]:
sol = plan.solve_task(task_name = "approach", q_init = q0_val)
# sol_list = plan.solve_task(task_name = ["approach","pickup"], q_init = q0_val)
# sol_list = plan.execute_plan(q_init = q0_val)


## Simulate plan execution


In [ ]:
robot_q0 = robot.get_initial_conditions[0:robot.ndof]
robot_qdot0 = robot.get_initial_conditions[robot.ndof:2*robot.ndof]

plan.simulate_plan(simulator = "bullet", q_init = robot_q0, qdot_init = robot_qdot0)